## GPU 스펙 확인

In [ ]:
!nvidia-smi

## 캐글 데이터 경로 확인

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 폴더 경로 설정

In [ ]:
workspace_path = '/kaggle/input/clouds-segmentation2024spring'  # 본인의 파일 경로 반영

In [ ]:
!pip install segmentation_models_pytorch

In [ ]:
!pip install albumentations==0.4.6
!pip install yacs

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import warnings
import random
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import torch
import segmentation_models_pytorch.utils as utils
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils import base
from segmentation_models_pytorch.utils import functional as F
from segmentation_models_pytorch.base.modules import Activation
from segmentation_models_pytorch.utils.metrics import IoU
import albumentations as albu
import torch.nn as nn
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.transforms as T

## patch

In [29]:
def save_patches(image, mask, patch_size, stride, save_dir, base_name):
    img_height, img_width = image.shape[:2]
    patch_id = 0

    for y in range(0, img_height - patch_size + 1, stride):
        for x in range(0, img_width - patch_size + 1, stride):
            img_patch = image[y:y + patch_size, x:x + patch_size]
            mask_patch = mask[y:y + patch_size, x:x + patch_size]

            img_patch_path = os.path.join(save_dir, 'ngr', f"{base_name}_{patch_id}.png")
            mask_patch_path = os.path.join(save_dir, 'label', f"{base_name}_{patch_id}.png")

            cv2.imwrite(img_patch_path, img_patch)
            cv2.imwrite(mask_patch_path, mask_patch)

            patch_id += 1

def patch(data_dir, patch_dir):
    patch_size = 224
    stride = 194
    save_dir = f'{patch_dir}/{patch_size}_{stride}'  # 패치가 저장될 디렉토리토리

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    if not os.path.exists(os.path.join(save_dir, 'label')):
        os.makedirs(os.path.join(save_dir, 'label'))
    if not os.path.exists(os.path.join(save_dir, 'ngr')):
        os.makedirs(os.path.join(save_dir, 'ngr'))

    image_files = [f for f in os.listdir(os.path.join(data_dir, 'ngr')) if f.endswith('.png')]
    mask_files = [f for f in os.listdir(os.path.join(data_dir, 'label')) if f.endswith('.png')]

    for img_file, mask_file in tqdm(zip(image_files, mask_files), total=len(image_files)):
        image = cv2.imread(os.path.join(data_dir, 'ngr', img_file))
        mask = cv2.imread(os.path.join(data_dir, 'label', mask_file))

        base_name = os.path.splitext(img_file)[0]
        save_patches(image, mask, patch_size, stride, save_dir, base_name)

In [ ]:
data_dir = f'{workspace_path}/train'

patch_dir = '/kaggle/working/cache'
os.makedirs(patch_dir, exist_ok=True)

patch(data_dir, patch_dir)

## seg_test_ensamble

In [ ]:
def load_model(model_path, device):
    model = torch.load(model_path)
    model = model.to(device)
    model.eval()
    return model

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=lambda x, **kwargs: x.transpose(2, 0, 1).astype('float32')),
    ]
    return albu.Compose(_transform)

def predict(model, image, preprocessing, device, patch_size=224, stride=97):
    height, width, _ = image.shape
    num_classes = 4  # 클래스 수
    output_mask = np.zeros((num_classes, height, width), dtype=np.float32)
    count_mask = np.zeros((height, width), dtype=np.float32)

    with torch.no_grad():
        for y in range(0, height - patch_size + 1, stride):
            for x in range(0, width - patch_size + 1, stride):
                patch = image[y:y + patch_size, x:x + patch_size]
                sample = preprocessing(image=patch)['image']
                sample = torch.from_numpy(sample).to(device).unsqueeze(0)
                prediction = model(sample)
                prediction = prediction.squeeze().cpu().numpy()
                
                output_mask[:, y:y + patch_size, x:x + patch_size] += prediction
                count_mask[y:y + patch_size, x:x + patch_size] += 1

        # Normalize the output mask by the count mask to get the average
        for c in range(num_classes):
            output_mask[c] /= count_mask

    return output_mask

def seg_test_ensamble(first_model_path, second_model_path, test_images_dir, device='cuda' if torch.cuda.is_available() else 'cpu'):
    encoder = 'efficientnet-b0'
    encoder_weights = 'imagenet'
    preprocessing_fn = smp.encoders.get_preprocessing_fn(encoder, encoder_weights)
    
    PALETTE = {
        (0, 0, 0): 0, # background
        (255, 0, 0): 1,  # thick_cloud
        (0, 255, 0): 2,  # thin_cloud
        (255, 255, 0): 3  # cloud_shadow
    }
    
    first_model = load_model(first_model_path, device)
    second_model = load_model(second_model_path, device)
    preprocessing = get_preprocessing(preprocessing_fn)

    test_images = os.listdir(test_images_dir)
    for image_name in tqdm(test_images):
        image_path = os.path.join(test_images_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        first_mask = predict(first_model, image, preprocessing, device)
        second_mask = predict(second_model, image, preprocessing, device)
        mask = (first_mask + second_mask) / 2
        
        mask = np.argmax(mask, axis=0)
        color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
        for color, class_idx in PALETTE.items():
            color_mask[mask == class_idx] = color
        mask = cv2.cvtColor(color_mask, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f'/kaggle/working/predicted_masks/{image_name}', mask)


In [ ]:
first_model_path = '/kaggle/working/ckpt/aug_224_194_best_model.pth'
second_model_path = '/kaggle/working/ckpt/non_aug_224_194_best_model.pth'
test_images_dir = f'{workspace_path}/test/ngr'
os.makedirs('/kaggle/working/predicted_masks', exist_ok=True)
seg_test_ensamble(first_model_path, second_model_path, test_images_dir)

## result_to_csv

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formatted
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

result_path = '/kaggle/working/predicted_masks'

test_label_file_list = os.listdir(result_path)
test_label_path_list = [os.path.join(result_path, x) for x in test_label_file_list]

rle_list = []
for file_path in test_label_path_list:
    img = cv2.imread(file_path)
    rle = mask2rle(img)
    rle_list.append(rle)
    
submission = pd.read_csv(f'{workspace_path}/sample_submission.csv')

submission['Image_Label'] = test_label_file_list
submission['EncodedPixels'] = rle_list

os.makedirs('/kaggle/working/csv', exist_ok=True)
submission.to_csv('/kaggle/working/csv/ensamble.csv', index=False)